### Anomaly Detection - Anomaly Scores  Calculations

This notebook shows how to calculate anomaly scores for anomaly detection

In [1]:
%reload_ext autoreload
%autoreload 2
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

import IPython, IPython.display, os, datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

print(f"Tensorflow Version {tf.__version__}, Keras Vesion: {keras.__version__}")

2023-03-05 21:15:13.365046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow Version 2.10.0, Keras Vesion: 2.10.0


error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fa564f3c640>, 'Connection to proxy-zsgov.external.lmco.com timed out. (connect timeout=15)'))


## Create Utility to compute anomaly score

In [5]:
%%writefile "ts_anom_utils.py"

# DO NOT EDIT THIS FILE - GENERATED FROM 06_anom_scores_calc.ipynb

# Study this carefully

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle


'''~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    This will compute the F-score given y and yhat
'''
def computeFScore(y, yhat):
    nu = np.sum( (y - yhat) ** 2)
    de = 1 or np.sum( (y-np.average(y))**2 )

    FSCORE=1-np.sqrt(nu/de)
    return np.round(FSCORE, 4)

'''~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    broken pairs columns are  "x, y, resid, fitness, threshhold, ranking"
    This will return all sensors whose standardized error exceeds threshhold value
'''
def get_brknpairs(r, fscore, standardized_err, errdf, thr=1.4, topn=10):
    r1  = round(r, 4)
    se  =  np.round(abs(standardized_err.loc[r.name]), 4)
    idx = abs(se.values).argsort()

    ret = [[c, c, r1[c], fscore[c], thr, se[c]] for c in se[idx][::-1][:topn].index if abs(se[c]) > thr ]
    top = [[c, se[c]] for c in se[idx][::-1][:topn].index]
    return  len(ret), ret, top

'''~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    compute Anomaly scores
    ydf: original Y Values
    pdf: predicted values

    You must always pass escaler unless you are using validation data to get the estimate of the error

    returns: 
        broken-pairs columns are  "x, y, resid, fitness, threshhold, ranking"
'''
def compute_scores(y: pd.DataFrame, yhat: pd.DataFrame, errorDF= None, escaler=None, top_n=15, fscore=None, file=None): 
    # 1. Compute the Absolute Error data frame
    resid = y - yhat
    error = abs(resid)

    # 2. Lets scale if scaler is not given => assuming this is training errors
    
    if ( escaler is None ):
        escaler = StandardScaler().fit(error)
        standardized_err = pd.DataFrame(escaler.transform(error), columns=error.columns, index=resid.index)

        # Compute the error statistics only if escaler is None
        fscore = computeFScore(y, yhat)

        errorDF = error.describe()
        errorDF.loc['fscore'] = fscore
        errorDF.loc['std_mean'] = standardized_err.mean()
        errorDF.loc['std_std'] = standardized_err.std()
    else:
        standardized_err = pd.DataFrame(escaler.transform(error), columns=error.columns, index=resid.index)


    # 4. Compute the Frobenius norm 
    score = np.linalg.norm(error.values ,axis=1)
    norm_score = np.linalg.norm(standardized_err ,axis=1)

    appl = resid.apply(get_brknpairs, args=( fscore, standardized_err, errorDF, 1.4, 20), axis=1)


    # 6. put them all together
    ret = pd.DataFrame(range(len(y)), columns=["line"], index=y.index)
    ret['time']             = y.index
    ret['score']            = score
    ret['norm_score']       = norm_score
    ret['numBroken']        = [c[0] for c  in appl.values]
    ret['brokenInvariants'] = [c[1] for c  in appl.values]
    ret['brokenSensors']    = [c[2] for c  in appl.values]

    if ( file is not None):
        errorDF.to_csv(f'{file}_errordf.csv')
        pickle.dump(escaler, open(f'{file}_escaler.pkl', 'wb'))

    return ret, error, standardized_err, errorDF, escaler, fscore

# The way how to use this 
'''
    Split data into train, validation, and test data
    Train on "training"
    Compute yhat on validation data
    call compute_scores on y and yhat of validation data.
    Use the escaler to detect anomalies on test data
    => COmpute the F-scrore, precision, recall scores
''';




Overwriting ts_anom_utils.py


### How to use the utility 

In [537]:
file="/tmp/"

# Pass file parameter only when you want to save the parameters
ret, error, standardized_err, errorDF, escaler, fscore = compute_scores(y, yhat, file=file)

suffix = str(ret.index[0]).replace(" ", '') + "--" + str(ret.index[-1]).replace(" ", '')
ret.to_csv(f'{file}SCORE-{suffix}.csv', index=False)


# Read back and call for future calls
file="/tmp/test_"
errorDF = pd.read_csv(f'{file}_errordf.csv', index_col=0)
fscore = errorDF.loc['fscore']
escaler= pickle.load(open(f'{file}_escaler.pkl', 'rb') )

ret


,line,time,score,norm_score,numBroken,brokenInvariants,brokenSensors
2021-01-01 00:00:00,0,2021-01-01 00:00:00,1.814307,1.513424,0,[],"[[B, 0.9705], [C, 0.8738], [A, 0.7648]]"
2021-01-01 01:00:00,1,2021-01-01 01:00:00,2.682773,1.155615,0,[],"[[C, 1.1281], [B, 0.1849], [A, 0.1692]]"
2021-01-01 02:00:00,2,2021-01-01 02:00:00,1.930179,1.558317,0,[],"[[C, 1.1099], [A, 0.9689], [B, 0.5077]]"
2021-01-01 03:00:00,3,2021-01-01 03:00:00,2.538528,2.438391,2,"[[B, B, -1.4621, -0.9597, 1.4, 1.6632], [A, A, 0.2494, -1.6261, 1.4, 1.5645]]","[[B, 1.6632], [A, 1.5645], [C, 0.8556]]"


## Anomaly score calculations explaination

Explaination of the code above with an exmaple

In [60]:
# Lets take a sample data and work through anomaly calculations
# We create a 
'''
    ydf => orignal values
    pdf => predicted value
    edf => error of the predictions
'''

np.random.seed(1)

rw = np.array([1,2,3])
ar = np.array([
    rw, rw + 1, rw + 2, rw*2
])
ydf = pd.DataFrame(ar, columns="A B C".split())
ydf.index = pd.date_range("2021-1-1", periods=len(ydf), freq="1H")

# Create a sample prediction data
pdf = pd.DataFrame(ydf.values + np.random.normal(size=ydf.values.shape), columns=ydf.columns)
pdf.index = ydf.index

# Compute the Errr
edf = ydf - pdf
edf.index = ydf.index

abs_edf = abs(edf)
abs_edf.index = ydf.index

sdf=pd.DataFrame(columns=["<->"], data=['<->']*len(ydf))
sdf.index = ydf.index
display(pd.concat([ydf, sdf, pdf, sdf, edf, sdf, abs_edf],  axis=1))

# This will describe your error distribution
edf.describe()

,A,B,C,<->,A,B,C,<->,A,B,C,<->,A,B,C
2021-01-01 00:00:00,1,2,3,<->,2.624345,1.388244,2.471828,<->,-1.624345,0.611756,0.528172,<->,1.624345,0.611756,0.528172
2021-01-01 01:00:00,2,3,4,<->,0.927031,3.865408,1.698461,<->,1.072969,-0.865408,2.301539,<->,1.072969,0.865408,2.301539
2021-01-01 02:00:00,3,4,5,<->,4.744812,3.238793,5.319039,<->,-1.744812,0.761207,-0.319039,<->,1.744812,0.761207,0.319039
2021-01-01 03:00:00,2,4,6,<->,1.750630,5.462108,3.939859,<->,0.249370,-1.462108,2.060141,<->,0.249370,1.462108,2.060141


,A,B,C
count,4.000000,4.000000,4.000000
mean,-0.511705,-0.238638,1.142703
std,1.396298,1.097357,1.251523
min,-1.744812,-1.462108,-0.319039
25%,-1.654462,-1.014583,0.316369
50%,-0.687487,-0.126826,1.294156
75%,0.455270,0.649119,2.120490
max,1.072969,0.761207,2.301539


In [37]:
# Lets compute the scale 
# you will save this errscale for future to validate test scores

errscale = StandardScaler().fit(abs_edf)
standardized_edf = pd.DataFrame( errscale.transform(abs_edf), columns=abs_edf.columns)
standardized_edf

,A,B,C
0,0.764827,-0.970549,-0.873801
1,-0.169247,-0.184940,1.128095
2,0.968906,-0.507671,-1.109884
3,-1.564486,1.663160,0.855589


In [38]:
top_n = 2   # How many top sensors you want -> if you have large data, you may want to choose 10
suspicious_sensors_idx = np.argsort(-standardized_edf.abs().values )[:,:top_n]
suspicious_sensors = [f for f in np.array(standardized_edf.columns)[suspicious_sensors_idx] ]

suspicious_sensors_idx, suspicious_sensors_idx.shape, suspicious_sensors


(array([[1, 2],
        [2, 1],
        [2, 0],
        [1, 0]]),
 (4, 2),
 [array(['B', 'C'], dtype=object),
  array(['C', 'B'], dtype=object),
  array(['C', 'A'], dtype=object),
  array(['B', 'A'], dtype=object)])

In [50]:
# this will give number of sensors deviated by more than by one SD
threshold = 1.1

vals = (standardized_edf.abs() > threshold)
np.count_nonzero(vals, axis=1)

array([0, 1, 1, 2])

In [234]:
score = np.linalg.norm(abs_edf.values ,axis=1)
norm_score = np.linalg.norm(standardized_edf.values ,axis=1)
score, norm_score

(array([1.8143068 , 2.68277327, 1.93017874, 2.53852811]),
 array([1.51342428, 1.15561513, 1.55831656, 2.43839088]))

In [235]:
# Combine them to get a score you can write 
z = zip(score, norm_score, suspicious_sensors)
adf=pd.DataFrame(z, columns="score norm_score suspicious_sensors".split())

In [236]:
# Finally create scores file and optinally saVE IT
scoreDF = pd.concat([adf, edf, standardized_edf], axis=1)
#scoreDF.to_csv("/tmp/temp.csv")
#pd.read_csv("/tmp/temp.csv", index_col=0)

,score,norm_score,suspicious_sensors,A,B,C,A.1,B.1,C.1
2021-01-01 00:00:00,1.814307,1.513424,['B' 'C' 'A'],-1.624345,0.611756,0.528172,0.764827,-0.970549,-0.873801
2021-01-01 01:00:00,2.682773,1.155615,['C' 'B' 'A'],1.072969,-0.865408,2.301539,-0.169247,-0.184940,1.128095
2021-01-01 02:00:00,1.930179,1.558317,['C' 'A' 'B'],-1.744812,0.761207,-0.319039,0.968906,-0.507671,-1.109884
2021-01-01 03:00:00,2.538528,2.438391,['B' 'A' 'C'],0.249370,-1.462108,2.060141,-1.564486,1.663160,0.855589


In [538]:
errorDF

,A,B,C
count,4.000000e+00,4.000000e+00,4.000000e+00
mean,1.172874e+00,9.251197e-01,1.302223e+00
std,6.816106e-01,3.728207e-01,1.022884e+00
min,2.493704e-01,6.117564e-01,3.190391e-01
25%,8.670691e-01,7.238443e-01,4.758886e-01
...,...,...,...
75%,1.654462e+00,1.014583e+00,2.120490e+00
max,1.744812e+00,1.462108e+00,2.301539e+00
fscore,-1.626100e+00,-9.597000e-01,-2.149900e+00
std_mean,1.110223e-16,5.551115e-17,1.110223e-16


In [1]:
# Create a function to call a URL and get results in python